In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:
n = 100_000
X = np.random.random_sample((n, 2))*2*np.pi - np.pi
y = np.sin(X[:, 0] + X[:, 1]).reshape(-1, 1)

In [ ]:
m = 10_000
X_test = np.random.random_sample((n, 2))*2*np.pi - np.pi
y_test = np.sin(X_test[:, 0] + X_test[:, 1]).reshape(-1, 1)

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')
ax.plot_trisurf(X[:, 0], X[: ,1], y.ravel())

In [ ]:
class FuncDataset(Dataset):

    def __init__(self, X, y):
        super().__init__()
        self.X = torch.from_numpy(X).float()
        self.y = torch.from_numpy(y).float()

    def __len__(self):
        return self.X.shape[0]
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [ ]:
dataset = FuncDataset(X, y)

In [ ]:
NUM_EPOCHS = 10
BATCH_SIZE = 32
ALPHA = 1e-3

In [ ]:
data_loader = DataLoader(dataset=dataset, batch_size=BATCH_SIZE, shuffle=True)

# Modello sequenziale

In [ ]:
NUM_EPOCHS = 10
BATCH_SIZE = 32
ALPHA = 1e-3

In [ ]:
IN = 2
OUT1 = 50
OUT2 = 1

In [ ]:
model = nn.Sequential(
    torch.nn.Linear(IN, OUT1),
    nn.ReLU(),
    torch.nn.Linear(OUT1, OUT2)
)

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=ALPHA)

In [ ]:
losses = []
for epoch in range(NUM_EPOCHS):
    for i, (x_batch, y_batch) in tqdm(enumerate(data_loader)):
        optimizer.zero_grad()
        y_ = model(x_batch)
        loss = F.mse_loss(y_batch, y_)
        losses.append(loss)
        loss.backward()
        optimizer.step()
        if i-1 % 1000 == 0:
            print(torch.mean(torch.tensor(losses[-1000:])))

# Modello funzionale

In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = torch.nn.Linear(IN, OUT1)
        self.linear2 = torch.nn.Linear(OUT1, OUT2)
        
    def forward(self, x):
        x = F.relu(self.linear1(x))
        y_ = self.linear2(x)
        return y_

In [ ]:
model = MLP() #.cuda()

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=ALPHA)

In [ ]:
losses = []
for epoch in range(NUM_EPOCHS):
    for i, (x_batch, y_batch) in tqdm(enumerate(data_loader)):
        optimizer.zero_grad()
        y_ = model(x_batch)
        loss = F.mse_loss(y_batch, y_)
        losses.append(loss)
        loss.backward()
        optimizer.step()
        if i-1 % 1000 == 0:
            print(torch.mean(torch.tensor(losses[-1000:])))